In [5]:
def get_general_dirichlet_domain(generator, base):
    
    n = len(base[0])
    G = PermutationGroup(generator)
    inequalities = []

    for i in range(n):
        
        #print('Estabilizando la componente',i+1)
        
        vector = base[i]
        partition = {}
        for j in range(n):
            if vector[j] in partition.keys():
                partition[vector[j]].append(j+1)
            else:
                partition[vector[j]] = [j+1]
        
        H = G
        for subset in partition:
            H = H.stabilizer(partition[subset], "OnSets")
            
        #print('La cardinalidad luego de estabilizar es de',H.cardinality())
        
        if H.cardinality() == G.cardinality():
            continue
        
        R = [x.Representative() for x in libgap.Group(G.gens()).RightCosets(libgap.Group(H.gens()))]
        
        #print('Se obtuvo el tranversal')
        
        for coset in R:
            lista = [0 for i in range(n+1)]
            for i in range(n):
                lista[i+1] = lista[i+1] + vector[i]
                lista[libgap.OnPoints(i+1, coset)] = lista[libgap.OnPoints(i+1, coset)] - vector[i]
            inequalities.append(tuple(lista))
            
            lista = [0 for i in range(n+1)]
            for i in range(n):
                lista[i+1] = lista[i+1] + vector[i]
                lista[libgap.OnPoints(i+1, libgap.Inverse(coset))] = lista[libgap.OnPoints(i+1, libgap.Inverse(coset))] - vector[i]
            inequalities.append(tuple(lista))
        G = H
        #print('Se completaron las desigualdades')
    #print('Se está creando el poliedro con las desigualdades, hay', len(inequalities), 'desigualdades')
    
    for i in range(n): #seteamos todas las variables >=0
        vector = [0 for i in range(n+1)]
        vector[i+1] = 1
        inequalities.append(tuple(vector))
    
    F = Polyhedron(ieqs = inequalities)
    return F

In [8]:
def get_rays_orbits(n, generador, rayos):
    
    I = IntegerVectorsModPermutationGroup(PermutationGroup(generador))
    orbitas = {}
    for i in range(len(rayos)):
        
        representante = I.retract(rayos[i])
        if (representante in orbitas.keys()) == True:
            orbitas[representante] = orbitas[representante] + [rayos[i]]
            continue
        
        orbitas[representante] = [rayos[i]]
            
    return orbitas

In [9]:
import random, numpy

def generate_bases(dim, n):
    #generate n bases of R^dim
    random.seed(33)
    base = [[[0 for i in range(dim)] for j in range(dim)] for k in range(n)]
    k = 0
    while k < n:
        for i in range(dim):
            for j in range(dim):
                base[k][i][j] = random.choice([0, 1])
        if abs(numpy.linalg.det(base[k])) >= 0.9:
            k = k + 1
    return base

def print_matrix(matrix):
    print(numpy.array(matrix))
    return

In [11]:
def experiment(generator, dim, num_bases):

    orbitas = []
    base = generate_bases(dim, num_bases)
    for i in range(num_bases):
        F = get_general_dirichlet_domain(generator, base[i])
        rayos = F.rays()
        orbitas.append(get_rays_orbits(dim, generator, rayos))
        largos = [len(orbitas[i][orbita]) for orbita in orbitas[i].keys()]
        largos.sort(reverse = True)
        print('Para la base',i+1,'Los largos de las órbitas son:', largos, 'y hay ', len(F.Hrepresentation()), 'desigualdades')

In [ ]:
n = 15
generador = DihedralGroup(n).gens()
experiment(generador, n, 20)